In [108]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

import pyrr
import itertools

# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


If `downsample` is not `None`, a random downsampling will be implemented. If `None`, the pipeline will use the voxel-based downsampling.

In [109]:
# mockname = 'test_kiwi_2'
# mockname = 'test_simple_combined'
# mockname = 'test_simple'
mockname = 'test_simple_below'
voxel_size = 0.05
downsample = None
# downsample = 0.4

Nleaves = 2
debug = True
# minpointPR = np.array([-1.00000334, -1.0051055, 1.90000071])
# maxpointPR = np.array([0.99999666, 1.0948945, 5.10000071])
minpointPR = None
maxpointPR = None

## Tree and leaves segmentation

Now we create the module to segmentate trees. This will be tuned acordingly for each data set, so below module only works for this particular data set.

In [110]:
def segtree(df, leaves, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 2, 2
    # dx, dy = 5, 5

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees


We segmentate the trees below,

In [111]:
# Conver `numpy` to `npy`
loads.numpy2npy(mockname)

s0100000.numpy done --> Number of beams: 360000


In [112]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
# round x, y, z to match voxel size significative figures
df[['x', 'y', 'z']] = df[['x', 'y', 'z']].round(2)
N = len(df)
print(N)

Number of files: 1
360000


In [113]:
# apply filter
if False:
    
    leaves = np.ones(len(df), dtype=bool)
    for i in ['x', 'y']:
        leaves &= (df[i] > -1) & (df[i] < 1)
    leaves &= (df['z'] > 1)

    df = df[leaves]

In [114]:

def voxel_subsampling(voxel_size, POINTS):

    nb_vox = np.ceil((np.max(POINTS, axis=0) - np.min(POINTS, axis=0))/voxel_size)
    ni, nj, nk = nb_vox
    print('min point:', np.min(POINTS, axis=0))
    print('max point:', np.max(POINTS, axis=0))
    print('Number of voxels: i:%d, j:%d, k:%d --> Total: %d' %(ni, nj, nk, np.product(nb_vox)))

    non_empty_voxel_keys, inverse, nb_pts_per_voxel = np.unique(((POINTS - np.min(POINTS, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted = np.argsort(inverse)
    print('Number of non-empty voxels: %d' %(len(non_empty_voxel_keys)))

    voxel_grid={}
    voxel_grid_ptsidx = {}
    grid_barycenter,grid_candidate_center = [], []
    last_seen=0

    for idx, vox in enumerate(non_empty_voxel_keys):

        idxs_per_vox = idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]
        voxel_grid[tuple(vox)] = POINTS[idxs_per_vox]
        voxel_grid_ptsidx[tuple(vox)] = idxs_per_vox

        # grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))

        idx_grid_candidate_center = np.linalg.norm(voxel_grid[tuple(vox)] - np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()
        grid_candidate_center.append(voxel_grid_ptsidx[tuple(vox)][idx_grid_candidate_center])

        last_seen+=nb_pts_per_voxel[idx]

        # if idx < 5:
        #     print('========')
        #     print(voxel_grid_ptsidx[tuple(vox)][idx_grid_candidate_center])

    # print('Downsampling percentage: %.1f %%' %(100 * len(grid_candidate_center) / len(POINTS)))
    # minpoint = np.min(POINTS, axis=0)

    # return list(grid_candidate_center) #, minpoint
    return list(grid_candidate_center)

# Sanity check
leaves = np.ones(len(df), dtype=bool)
for i in ['x', 'y']:
    leaves &= (df[i] > -1) & (df[i] < 1)
leaves &= (df['z'] > 1)

_POINTS = df[['x', 'y', 'z']][leaves].to_numpy()
vox, inverse, nb_pts_per_voxel = np.unique((np.round(_POINTS - np.min(_POINTS, axis=0), 10) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)

for i in set(vox[:,2]):
    print('K:',i, '-- Non empty voxels -->', np.sum(vox[:,2] == i))

K: 0 -- Non empty voxels --> 1600
K: 59 -- Non empty voxels --> 1600
K: 19 -- Non empty voxels --> 1600
K: 39 -- Non empty voxels --> 1600


In [115]:
def random_downsample(N, downsample):

    resdir = os.path.join(_data, mockname, 'random_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        idx = np.load(outdir)

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, N, int(N * downsample))
        # inds = np.zeros(N, dtype=bool)
        # inds[idx] = True

        np.save(outdir, idx)

    return idx

### Implement and keep Downsampled points

Below code will implement a downsampling using either `random` or `voxel`. The donsampling is performed by saving the corresponding indexes list of the downsampled percentage from the original data size. If index list already exists we just take it to make the downsampling, if it does not exist yet, we created and save ir under its corresponding directory.

In [116]:

if downsample is not None:
    inds = random_downsample(N, downsample)
    print('Random downsampling...')
else:
    inds = voxel_subsampling(voxel_size, df[['x', 'y', 'z']].to_numpy())
    print('Voxel downsampling...')

print('Downsampling percentage: %.1f %%' %(100 *  len(inds) / len(df['x'])))

df = df.iloc[inds]
POINTS = df[['x', 'y', 'z']].to_numpy()
SENSORS = df[['sx', 'sy', 'sz']].to_numpy()

# Compute lower point
minpoint = np.min(POINTS, axis=0) # check this...
print('minpoint:', minpoint)


min point: [-11.  -5.   0.]
max point: [10.  5. 25.]
Number of voxels: i:420, j:200, k:500 --> Total: 42000000
Number of non-empty voxels: 73473
Voxel downsampling...
Downsampling percentage: 20.4 %
minpoint: [-11.  -5.   0.]


In [117]:
leaves = np.ones(len(df), dtype=bool)
for i in ['x', 'y']:
    leaves &= (df[i] > -1) & (df[i] < 1)
leaves &= (df['z'] > 1)

# print(len(df), np.sum(leaves))
# loads.showPCfromDF(df[['x', 'y', 'z']][leaves])

### leave and tree segmentation

In [118]:
# extract leaves. Boolean array output
# leaves = loads.extract_leaves(df, show=True)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df, leaves, show=False)

### Second downsampling: keep only points that colide with Plant Region

In [119]:
inPR = (leaves) & (trees['tree_0'])
minBB, maxBB = np.min(POINTS[inPR.values], axis=0), np.max(POINTS[inPR.values], axis=0)

# if debug:
#     minBB[2] = minBB[2] - 0*voxel_size
#     maxBB[2] = maxBB[2] + 0*voxel_size

# Make sure Plant Region min & max points are multiples of voxel size
# to match first voxelization where we implemented the downsampling
if minpointPR is None:
    minpointPR = minpoint + np.floor(np.abs(minpoint - minBB)/voxel_size) * voxel_size
if maxpointPR is None:
    maxpointPR = minpoint + np.ceil(np.abs(minpoint - maxBB)/voxel_size) * voxel_size
# else:
#     minpointPR = minBB
#     maxpointPR = maxBB

print(minpointPR, maxpointPR)

boxPR = pyrr.aabb.create_from_bounds(minpointPR, maxpointPR)

lines = np.stack((POINTS, SENSORS), axis=1)
f = lambda line: pyrr.geometric_tests.ray_intersect_aabb(pyrr.ray.create_from_line(line), boxPR) is not None
# res = np.array(list(map(f, lines)))
res = []

for num, line in enumerate(lines):

    try:
        res.append(f(line))
    except Exception as e:
        res.append(False)
        print('Error:', e)
        print('On line:',num)

res = np.array(res)

[-1. -1.  2.] [1. 1. 5.]
Error: min() arg is an empty sequence
On line: 73472


In [120]:
POINTS, SENSORS = POINTS[res], SENSORS[res]

leaves = leaves[res]

for key, val in trees.items():
    trees[key] = val[res]

In [121]:
# save indexes of voxel-based downsample

idxs = np.array(inds)[res]

if downsample is not None:
    dirname = 'random_%s' %(str(downsample))
    resdir = os.path.join(_data, mockname, dirname, 'lad_%s' %(str(voxel_size)))
else:
    dirname = 'voxel'
    resdir = os.path.join(_data, mockname, dirname, 'lad_%s' %(str(voxel_size)))

if not os.path.exists(resdir): os.makedirs(resdir)
outdir = os.path.join(resdir, 'inds.npy')
np.save(outdir, idxs)

## Ray tracing

In [122]:
sample = None

inPR = (leaves) & (trees['tree_0'])

if sample is not None:
    print('# iter...', len(POINTS[::sample]))
    m3s = rayt.main2(POINTS[::sample], SENSORS[::sample], POINTS[inPR], voxel_size, resdir, 'tree_0', (minpointPR, maxpointPR), show=True)
else:
    print('# iter...', len(POINTS))
    print('Results will be saved at %s' %(resdir))
    print('-------------')
    m3s = rayt.main2(POINTS, SENSORS, POINTS[inPR], voxel_size, resdir, 'tree_0', (minpointPR, maxpointPR), show=False)

# iter... 11344
Results will be saved at /Users/omar/projects/planttech/data/test_simple_below/voxel/lad_0.05
-------------


11344it [03:18, 57.01it/s] 

tot vox: 	 97600
voxels hitted: 	 59374
Percentage of voxels hitted by beam: 0.61
voxels hitted (OLD): 	 0
Percentage of voxels hitted by beam (OLD): 0.00


In [ ]:
inPR = (leaves) & (trees['tree_0'])
# resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))
NI, N0, NP0, NP = lad.get_attributes_per_k(POINTS[inPR], voxel_size, (minpointPR, maxpointPR), 'tree_0', kval=2, resdir=resdir, showall=True)

## LIA

In [19]:
def best_fit_lia(mockname, Nleaves):

    df = loads.npy2pandas(mockname)
    # extract leaves. Boolean array output
    
    # leaves = loads.extract_leaves(df, show=False)
    leaves = np.ones(len(df), dtype=bool)
    for i in ['x', 'y']:
        leaves &= (df[i] > -1) & (df[i] < 1)

    # extract trees. Dictionary with boolean arrays output
    trees = segtree(df, leaves, show=False)

    for key, val in trees.items():

        keep = (val) & (leaves) # take the LPC per tree
        points = df[['x', 'y', 'z']].to_numpy()[keep]

        res = lia.bestfit_pars_la(points, mockname, Nleaves, treename=key)
        lia.best_fit_pars_plot(res, key, mockname)

In [20]:
def get_lia(mockname):

    df = loads.npy2pandas(mockname)

    # extract leaves. Boolean array output

    # leaves = loads.extract_leaves(df, show=False)
    leaves = np.ones(len(df), dtype=bool)
    for i in ['x', 'y']:
        leaves &= (df[i] > -1) & (df[i] < 1)

    # extract trees. Dictionary with boolean arrays output
    trees = segtree(df, leaves, show=False)


    # load bestfit results
    for key, val in trees.items():

        keep = (val) & (leaves)
        print(sum(keep), len(keep))
        points = df[['x', 'y', 'z']].to_numpy()[keep]
    
        bestfit_file = os.path.join(_data, mockname, 'lia', 'bestfit_%s.npy' %(key))
        res = np.load(bestfit_file, allow_pickle=True)
        res = res.tolist()

        text = 'leaf area=%.2f \n %s=%.4f \n %s=%.4f \n %s=%.4f ' %(res['leafsize'], 'voxel_size_w', res['voxel_size_w_bestfit'],'kd3_sr', res['kd3_sr_bestfit'],'max_nn', res['max_nn_bestfit'])
        print(text)

        chi2 = lia.leaf_angle(points, mockname, key, res['voxel_size_w_bestfit'], 
                                res['kd3_sr_bestfit'], res['max_nn_bestfit'], save=True,
                                    savefig=True, text=text, voxel_size_h=0.1)

        # save indexes from main df
        # inds = np.where((val) & (leaves))
        np.save(os.path.join(_data, mockname, 'lia', 'inds.npy'), keep)


In [21]:
# Check we have a mesh
if False:
    meshfile = os.path.join(_data, mockname, 'mesh.ply')
    lad.see_mesh(meshfile)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [22]:
best_fit_lia(mockname, Nleaves)

Number of files: 1
voxel_size_w 0.0001 DONE...
voxel_size_w 0.001 DONE...
voxel_size_w 0.01 DONE...
voxel_size_w 0.1 DONE...
voxel_size_w 1 DONE...
voxel_size_w BESTFIT:	 0.0001
kd3_sr 0.001 DONE...
kd3_sr 0.01 DONE...
kd3_sr 0.1 DONE...
kd3_sr 1.0 DONE...
kd3_sr BESTFIT:	 0.001
max_nn 3 DONE...
max_nn 5 DONE...
max_nn 10 DONE...
max_nn 20 DONE...
max_nn 50 DONE...
max_nn 100 DONE...
max_nn BESTFIT:	 10


In [23]:
get_lia(mockname)

Number of files: 1
68162 360000
leaf area=4.00 
 voxel_size_w=0.0001 
 kd3_sr=0.0010 
 max_nn=10.0000 


In [123]:
def runall(pointsPR, sensorsPR, inPR, voxel_size, tree, N, PRbounds, resdir, kbins=None):

    # resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))
    inds_file = os.path.join(resdir, 'inds.npy')
    inds0 = np.load(inds_file)

    # resdir = os.path.join(_data, mockname, 'lad_%s' %(str(voxel_size)))

    inds_lia = np.load(os.path.join(_data, mockname, 'lia', 'inds.npy'))

    isfigures = os.path.join(resdir, 'figures')
    if not os.path.exists(isfigures):
        os.makedirs(isfigures)

    attributes2_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(tree, str(voxel_size)))
    if os.path.isfile(attributes2_file):
        m3b = np.load(attributes2_file)

    print('voxel_size:', voxel_size)

    # m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, tree, PRbounds)

    # _,_,_, m3scount = lad.density_counts(pointsPR, voxel_size)

    # Load LIAs and its weights saved at `get_lia()`.
    # Size of lias and ws arrays is the original size after leaf and tree extraction only.
    lias, ws = loads.load_lias_ws(mockname, 'tree_0')

    # Create  arrays of original size filled with -99
    lias0 = np.full(N, -99)
    ws0 = np.full(N, -99)

    # fill arrays with lias and ws values where it correspond to.
    lias0[np.where(inds_lia)[0]] = lias
    ws0[np.where(inds_lia)[0]] = ws

    # Finally, apply downsampling and second dowsampling to lias and ws.
    lias = lias0[inds0[inPR]]
    ws = ws0[inds0[inPR]]

    try:
        assert len(lias) == sum(inPR)
    except Exception as e:
        print('lias size does not match with Plant Region size.')
        print(e)

    try:
        assert len(ws) == sum(inPR)
    except Exception as e:
        print('ws size does not match with Plant Region size.')
        print(e)

    voxk = lad.get_voxk(pointsPR, PRbounds, voxel_size)
    bia = lad.get_bia(pointsPR, sensorsPR)
    meshfile = lad.get_meshfile(mockname)

    # print('----- DEBUG -----')
    # print(len(lias), len(ws), len(voxk))

    figext = '%s_%s' %(tree, str(voxel_size))
    # figext = None
    
    alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, meshfile, figext=figext, 
                            klia=False, use_true_lia=True)

    kmax = m3b.shape[2]
    
    if kbins is None:
        kbins = int(kmax/15)
    print('kmax', kmax)
    print('kbins', kbins)

    # Attribute 2 counts per voxel
    # outdir_count = os.path.join(resdir, 'm3count_%s_%s.npy' %(tree, str(voxel_size)))

    oldlad = True
    
    lads_mid_1, clai_1 = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6], PRbounds, tree, resdir, oldlad=oldlad, C=1)
    # lads_mid_05, clai_05 = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6], PRbounds, tree, resdir, oldlad=True, C=0.5)
    lads_0, clai_0 = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6]*0+1, PRbounds, tree, resdir, oldlad=oldlad, C=1)

    # lads_mid_old, _ = lad.get_LADS2(pointsPR, kmax, voxel_size, kbins, alphas_k[:,6], PRbounds, tree, resdir, oldlad=True)
    # lads_mid_old = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], alpha2=1)
    lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax, PRbounds)

    # lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0, 'Correction Weights':lads_mid_w}#, 'Correction counts':lads_mid_counts}
    # lads = {'Truth':lads_mesh, 'Correction Mean C=1':lads_mid_1, 'Correction Mean C=0.5':lads_mid_05,}
    lads = {'Truth':lads_mesh, 'Correction Mean C=1':lads_mid_1, 'No Correction C=1':lads_0}
    # clai = lad.get_clai(m3att, alphas_k)
    attributes_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(tree, str(voxel_size)))
    if os.path.isfile(attributes_file):
        RT = 'Y'
    else:
        RT = 'N'
        
    # text = {'tree':tree, 'VS':voxel_size, 'RT':RT, 'CLAI 1.0':np.round(clai_1, 3),  'CLAI 0.5':np.round(clai_05, 3)}
    text = {'tree':tree, 'VS':voxel_size, 'RT':RT, 'CLAI 1.0':np.round(clai_1, 3),  'CLAI 1.0 -- NC':np.round(clai_0, 3)}
    txt = []
    for key, val in text.items():
        txt.append('%s=%s \n' %(key, str(val)))
    text = (' ').join(txt)

    savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
    figures.plot_lads(lads, text, savefig=savefig)


In [124]:
inPR = (leaves) & (trees['tree_0'])
runall(POINTS[inPR], SENSORS[inPR], inPR, voxel_size, 'tree_0', N, (minpointPR, maxpointPR), resdir, kbins=4)

voxel_size: 0.05
kmax 61
kbins 4
======= K: 0 =======
	 nI0: 1474.00
	 nI: 6.83
	 nP0: 13.17
	 nP: 40.00
======= K: 1 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 168.00
======= K: 2 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 352.00
======= K: 3 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 508.00
======= K: 4 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 707.00
======= K: 5 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 865.00
======= K: 6 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 1039.00
======= K: 7 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 1224.00
======= K: 8 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 1382.00
======= K: 9 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 1556.00
======= K: 10 =======
	 nI0: 99.00
	 nI: 0.00
	 nP0: 99.00
	 nP: 1425.00
======= K: 11 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 1513.00
======= K: 12 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.00
	 nP: 1529.00
======= K: 13 =======
	 nI0: 0.00
	 nI: 0.00
	 nP0: 0.0

In [ ]:
import open3d as o3d

meshfile = os.path.join(_data, mockname, 'mesh.ply')
mesh = o3d.io.read_triangle_mesh(meshfile)
vert = np.asarray(mesh.vertices)
tri = np.asarray(mesh.triangles)

print(np.array(mesh.vertices))